In [217]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [218]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, GlobalMaxPooling1D, Dropout 

### **Read Dataset**

In [219]:
df_train = pd.read_excel('train.xlsx')
df_test = pd.read_excel('test.xlsx')

In [220]:
df_train.head()

,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11
0,با زدن کد دستوری 223 10 اهنگ پیشواز لغو ن...,\nسلام. من مشترک ۹۱۲ هستم. برای فعال کردن آوای...,سلام علیکم آقا خسته نباشید. میخواستم یک کد اوا...,من دو سه روزه پیشوازمو زدم و فعال کردمش امروز ...,سلام صبح بخیر. ببخشید مزاحم میشم من لیست آوای ...,با چه کدی می تونم کد پیش صدا خط خودمو بگیرم,آقا من از یکی پرسیدم این آهنگی که میخونه پشت خ...,ببخشیدمن دوباره تماس میگیرم. قبلا هم پیام داده...,باید چه کدی رو بزنم که آوای انتظارم دیگه تمدید...,برای جایگزین کردن اهنگ پشت سیمکارتی چه شماره ...,من قبل از پونزده روز آوای انتظارم رو غیرفعال ک...
1,سلام طاعات و عباداتتون بخیر باشه من میخوام آوا...,از همراه من می خواستم اهنگ پیشواز فعال کنم ام...,من توی عوض کردن آوا مشکل دارم حالا به نظرتون ر...,اوای انتظار فعاله اما بهم زنگ می زنن بوق میخوره,سلام. وقتتون به خیر. صبح تماس گرفتم برام لینک ...,کد اهنگ پشت خطیمو میخوام بدونم,برای دریافت کد پیش صدا خط دوستم از چه روشی می...,چه کدی بزنم اوای انتظارم فعال بشه,آقا این خواننده پیشواز ما برداشته یه حرکت جلف ...,چکار کنم بهم زنگ می زنند پیش اواز جدید پخش بشه,میشه یه مدت پیشوازمو تعلیق کنم
2,سلام وقتتون بخیر داداش عزیزم برای غیرفعالسازی ...,کد زدم اهنگ پیشواز رو فعال کنم اما اهنگ پیشوا...,هر کاری می کنم اوای انتظار تغییر نمی کنه,سلام من آهنگ پیشواز رو فعال کردم ولی آهنگ پخش...,شعر ببار ای بارون ببار,میخام کد اهنگ پشت خطی خودمو بگیرم چکار کنم,کد اهنگ پشت خط همسرمو خام ایمو ارسال کنید,برای فعال کردن اهنگ پشت خطی روی خط خود از چه ...,نمیخوام پیش صدایی که روی خطم فعاله خودش تمدید...,سلام خسته نباشید من موقع فعالسازی آوای انتظار ...,هوا چطوره
3,سلام خسته نباشید. آوای انتظارم رو غیر فعال کرد...,میخوام اهنگ پیشواز فعال کنم اما از هیچ روشی ن...,سلام منو راهنمایی کنید یه کدیو اشتباه زدم یه آ...,داداشم اوای انتظار فعال کرده ولی اوای انتظارش...,سلام وقت بخیر آقا این لیست آوای انتظار ۱۴۰۲ ر...,اهنگ پشت خطی خط خودمو میخام چه کدی باید بزنم,از چه روشی می توان کد اوای انتظار خط همسایه ر...,اوای انتظارمو که غیرفعال کردم چجوری دوباره فع...,چکار کنم بهم زنگ می زنند پیش اهنگ پخش نشه,تعویضش کن,وقتتون بخیر باشه این آوای انتظاری که من انتخاب...
4,میخواستم اهنگ پیشوازمو لغو کنم برا همین ک...,پیام دادم به 8989 اما فعال نشد,سلام و عصر بخیر آقا. من می‌خواستم آهنگ ا...,از همراه من اهنگ پیشواز فعال کردم هزینش کم شد...,اهنگ پیشواز خوشحالم,کد پیش اهنگی رو خطمه چنده,کد اهنگ خط همسرم رو میخوام,چه جوری میتونم خوشه چین بنان رو بذارم آوای انت...,سلام و صبح بخیر. من می‌خواستم ببنم چطوری...,شیوه جایگزین کردن اهنگ پشت خطی,من دنبال یک رستوران اسپانیایی هستم


In [221]:
print(df_train.shape)
print(df_test.shape)

(340, 11)
(60, 11)


In [222]:
df_train.iloc[0]

class1          با زدن کد دستوری 223 10 اهنگ پیشواز لغو ن...
class2     \nسلام. من مشترک ۹۱۲ هستم. برای فعال کردن آوای...
class3     سلام علیکم آقا خسته نباشید. میخواستم یک کد اوا...
class4     من دو سه روزه پیشوازمو زدم و فعال کردمش امروز ...
class5     سلام صبح بخیر. ببخشید مزاحم میشم من لیست آوای ...
class6           با چه کدی می تونم کد پیش صدا خط خودمو بگیرم
class7     آقا من از یکی پرسیدم این آهنگی که میخونه پشت خ...
class8     ببخشیدمن دوباره تماس میگیرم. قبلا هم پیام داده...
class9     باید چه کدی رو بزنم که آوای انتظارم دیگه تمدید...
class10     برای جایگزین کردن اهنگ پشت سیمکارتی چه شماره ...
class11    من قبل از پونزده روز آوای انتظارم رو غیرفعال ک...
Name: 0, dtype: object

In [223]:
classes = [
        'مشکل غیرفعالسازی آوای انتظار',
        'مشکل فعالسازی آوای انتظار',
        'مشکل تغییر آوای انتظار',
        'مشکل پخش آوای انتظار',
        'لیست آواهای انتظار',
        'دریافت کد آوای انتظار خط خود',
        'دریافت کد آوای انتظار خط دیگران',
        'روش فعالسازی آوای انتظار',
        'روش غیرفعالسازی آوای انتظار',
        'روش تغییرآوای انتظار',
        'خارج از دامنه'
]

In [224]:
def create_dataset(df):
  output = {
        "text": [],
        "category": []
    }

  for data in df.iloc:
    for i,d in enumerate(data):
      output["text"].append(d)
      output["category"].append(classes[i])
   
  return output

In [225]:
train_data = create_dataset(df_train)
test_data = create_dataset(df_test)

In [226]:
print(len(train_data["text"]))
print(len(train_data["category"]))

3740
3740


In [227]:
print(train_data["text"][0])
print(train_data["category"][0])

     با زدن کد دستوری 223 10 اهنگ پیشواز لغو نشد چجوری اهنگ پیشواز رو لغو کنم
مشکل غیرفعالسازی آوای انتظار


In [228]:
train_dataset = pd.DataFrame(train_data)
test_dataset = pd.DataFrame(test_data)

In [229]:
train_dataset.head()

,text,category
0,با زدن کد دستوری 223 10 اهنگ پیشواز لغو ن...,مشکل غیرفعالسازی آوای انتظار
1,\nسلام. من مشترک ۹۱۲ هستم. برای فعال کردن آوای...,مشکل فعالسازی آوای انتظار
2,سلام علیکم آقا خسته نباشید. میخواستم یک کد اوا...,مشکل تغییر آوای انتظار
3,من دو سه روزه پیشوازمو زدم و فعال کردمش امروز ...,مشکل پخش آوای انتظار
4,سلام صبح بخیر. ببخشید مزاحم میشم من لیست آوای ...,لیست آواهای انتظار


In [230]:
train_dataset.shape

(3740, 2)

In [231]:
test_dataset.head()

,text,category
0,سلام خانم، من یه مشکلی با آوای انتظار دارم، چ...,مشکل غیرفعالسازی آوای انتظار
1,برای فعال کردن اهنگ پیشواز از طریق همراه من ا...,مشکل فعالسازی آوای انتظار
2,علو سلام مادر جون. خوبی عزیزم عیدت مبارک باشه ...,مشکل تغییر آوای انتظار
3,اوای انتظار فعال کردم پولشم کم شده اما زنگ می...,مشکل پخش آوای انتظار
4,کد پیشواز از بنان,لیست آواهای انتظار


In [232]:
test_dataset.shape

(660, 2)

In [233]:
train_dataset['category'].value_counts()

مشکل غیرفعالسازی آوای انتظار       340
مشکل فعالسازی آوای انتظار          340
مشکل تغییر آوای انتظار             340
مشکل پخش آوای انتظار               340
لیست آواهای انتظار                 340
دریافت کد آوای انتظار خط خود       340
دریافت کد آوای انتظار خط دیگران    340
روش فعالسازی آوای انتظار           340
روش غیرفعالسازی آوای انتظار        340
روش تغییرآوای انتظار               340
خارج از دامنه                      340
Name: category, dtype: int64

In [237]:
#delete NaN
train_dataset = train_dataset.dropna()
test_dataset = test_dataset.dropna()

In [238]:
print(train_dataset.shape)
print(test_dataset.shape)

(2761, 2)
(484, 2)


**stopword**

In [246]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [247]:
stopwords = read_stop_words('stopwords.txt')
print(len(stopwords))

1327


In [248]:
# install hazm library
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [249]:
from hazm import word_tokenize

def clean_data(dataset):
  text = dataset['text'].values
  text_tokenized = [word_tokenize(t) for t in text]
  text_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in text_tokenized]
  sentence = [' '.join(sentence) for sentence in text_tokenized_filtered]
  # remove numbers and white spaces
  sentence = [s.replace('\u200c', '') for s in sentence]
  sentence = [re.sub('[0-9]+', 'عدد', s) for s in sentence]
  sentence = [re.sub('\s+', ' ', s) for s in sentence]
  return sentence

In [250]:
clean_train_data = clean_data(train_dataset)
clean_test_data = clean_data(test_dataset)

In [251]:
clean_train_data[0]

'زدن کد دستوری عدد عدد اهنگ پیشواز لغو نشد چجوری اهنگ پیشواز لغو'

In [252]:
clean_test_data[0]

'خانم مشکلی آوای انتظار فعالش میخوام غیرفعالش هچین گزینهای پنل کاربریم ندارم ممنون میشم راهنماییم'

### **one hot Y**

In [253]:
def prepare_y(dataset):
  Y = pd.get_dummies(dataset['category']).values
  return Y

In [254]:
category_types = [
        'خارج از دامنه',
        'دریافت کد آوای انتظار خط خود',
        'دریافت کد آوای انتظار خط دیگران',
        'روش تغییرآوای انتظار',
        'روش غیرفعالسازی آوای انتظار',
        'روش فعالسازی آوای انتظار',
        'لیست آواهای انتظار',
        'مشکل تغییر آوای انتظار',
        'مشکل غیرفعالسازی آوای انتظار',
        'مشکل فعالسازی آوای انتظار',
        'مشکل پخش آوای انتظار'
]

In [255]:
category_types

['خارج از دامنه',
 'دریافت کد آوای انتظار خط خود',
 'دریافت کد آوای انتظار خط دیگران',
 'روش تغییرآوای انتظار',
 'روش غیرفعالسازی آوای انتظار',
 'روش فعالسازی آوای انتظار',
 'لیست آواهای انتظار',
 'مشکل تغییر آوای انتظار',
 'مشکل غیرفعالسازی آوای انتظار',
 'مشکل فعالسازی آوای انتظار',
 'مشکل پخش آوای انتظار']

### **Train & Test Split**

In [256]:
X_train = clean_train_data
X_test =  clean_test_data
y_train = prepare_y(train_dataset)
y_test = prepare_y(test_dataset)

In [257]:
print(np.array(X_train).shape)
print(np.array(X_test).shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)

(2761,)
(484,)
(2761, 11)
(484, 11)


### **Train The Model**

In [258]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [259]:
from transformers import AutoTokenizer,TFBertModel
MODEL_NAME = "HooshvareLab/bert-fa-zwnj-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert = TFBertModel.from_pretrained(MODEL_NAME)

Some layers from the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [260]:
bert.summary()

Model: "tf_bert_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  118297344 
                                                                 
Total params: 118,297,344
Trainable params: 118,297,344
Non-trainable params: 0
_________________________________________________________________


In [261]:
x_train = tokenizer(
    text=X_train,
    add_special_tokens=True,
    max_length=60,
    truncation=True,
    padding="max_length",
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

x_test = tokenizer(
    text=X_test,
    add_special_tokens=True,
    max_length=60,
    truncation=True,
    padding="max_length",
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [262]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [263]:
input_ids[0]

<tf.Tensor: shape=(60,), dtype=int32, numpy=
array([    2,  4309,  3441, 12342,  4420,  4420,  4265, 36379,  6009,
        4921, 16097,  4265, 36379,  6009,     3,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0], dtype=int32)>

In [264]:
input_ids[0].shape

TensorShape([60])

In [265]:
attention_mask[0]

<tf.Tensor: shape=(60,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>

In [266]:
attention_mask[0].shape

TensorShape([60])

In [273]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [274]:
max_len = 60
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids, attention_mask=input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation = 'relu')(out)
y = Dense(len(category_types), activation='softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [276]:
optimizer = Adam(
    learning_rate=5e-05, 
    epsilon=1e-08,
    clipnorm=1.0)

In [277]:

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [278]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 60)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 60)]         0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  118297344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 60,                                         

In [279]:
history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
    epochs=20,
    batch_size=36
)

Epoch 1/20


77/77 [==============================] - 103s 591ms/step - loss: 2.4268 - accuracy: 0.1000 - val_loss: 2.6308 - val_accuracy: 0.0868
Epoch 2/20
77/77 [==============================] - 38s 495ms/step - loss: 2.4014 - accuracy: 0.0953 - val_loss: 2.6623 - val_accuracy: 0.0868
Epoch 3/20
77/77 [==============================] - 36s 471ms/step - loss: 2.3981 - accuracy: 0.0873 - val_loss: 2.6291 - val_accuracy: 0.0868
Epoch 4/20
77/77 [==============================] - 36s 462ms/step - loss: 2.3958 - accuracy: 0.0942 - val_loss: 2.6142 - val_accuracy: 0.0868
Epoch 5/20
77/77 [==============================] - 37s 481ms/step - loss: 2.3960 - accuracy: 0.1047 - val_loss: 2.6308 - val_accuracy: 0.0868
Epoch 6/20
77/77 [==============================] - 37s 479ms/step - loss: 2.3941 - accuracy: 0.0996 - val_loss: 2.6413 - val_accuracy: 0.0868
Epoch 7/20
77/77 [==============================] - 36s 463ms/step - loss: 2.3963 - accuracy: 0.0982 - val_loss: 2.6241 - val_accuracy: 0.0868
Epoch 8/2

In [215]:
model.save('model.h5')

In [216]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

16/16 [==============================] - 6s 117ms/step


array([0.0934613 , 0.07802659, 0.09379119, 0.09766004, 0.10065686,
       0.09261222, 0.1073371 , 0.05654095, 0.05607916, 0.14965174,
       0.07418285], dtype=float32)